In [14]:
import pandas as pd
import datetime 
import threading
import concurrent.futures

from tqdm import tqdm 

In [2]:
stop_times = pd.read_csv('stop_times.txt', sep=',')
stop_times['arrival_time'] = pd.to_datetime(stop_times['arrival_time'])

print(stop_times.head())

     trip_id        arrival_time departure_time   stop_id  stop_sequence
0  1012-10-0 2023-03-02 18:00:00       18:00:00    301729              1
1  1012-10-0 2023-03-02 18:01:54       18:01:54    301764              2
2  1012-10-0 2023-03-02 18:03:48       18:03:48    301724              3
3  1012-10-0 2023-03-02 18:05:42       18:05:42    301730              4
4  1012-10-0 2023-03-02 18:07:36       18:07:36  30003042              5


In [5]:
frequencies = pd.read_csv('frequencies.txt', sep=',')
frequencies['start_time'] = pd.to_datetime(frequencies['start_time'])
# frequencies = frequencies[(frequencies['trip_id'] == '609F-21-0')].reset_index()
print(frequencies.count())

trip_id         39786
start_time      39786
end_time        39786
headway_secs    39786
dtype: int64


In [15]:
# partidas = pd.DataFrame({'departure_id': [], 'trip_id': [], 'departure_time': []})

# def criacaoPartidas(df):
#     for index, row in df.iterrows():
#         time = 0
#         timef = row['headway_secs']
#         while time < 3600:
#             dt = datetime.timedelta(seconds=time)
#             ref_dt = row['start_time'] + dt
#             hour = str(ref_dt.hour)
#             min = str(ref_dt.minute)
            
#             new_partida = pd.DataFrame({'departure_id': [row['trip_id'] + '_' + hour +':'+ min],
#                                         'trip_id':[row['trip_id']],
#                                         'departure_time': [row['start_time'] + dt] })
            
#             #print(new_partida)
#             partidas= pd.concat([partidas,new_partida],ignore_index=True)
            
#             time += timef
            
#     return new_partida['departure_id'].count()

In [37]:
partidas = {'departure_id': [], 'trip_id': [], 'departure_time': []}

for i in tqdm(range(len(frequencies))):
    time = 0
    row = frequencies.iloc[i]
    timef = int(row['headway_secs'])
    while time < 3600:
        dt = timedelta(seconds=time)
        ref_dt = row['start_time'] + dt
        hour = str(ref_dt.hour)
        min = str(ref_dt.minute)
        
        partidas['departure_id'].append(row['trip_id'] + '_' + hour +':'+ min)
        partidas['trip_id'].append(row['trip_id'])
        partidas['departure_time'].append(row['start_time'] + dt)
        
        time += timef

partidas = pd.DataFrame(partidas)
print(partidas['departure_id'].count())

100%|███████████████████████████████████| 39786/39786 [00:07<00:00, 5222.46it/s]


172844


In [80]:
partidas.to_csv("partidas.txt")

In [ ]:
""" listaTrip_id = frequencies.groupby('trip_id', as_index=False).count()
listaTrip_id = listaTrip_id['trip_id']
listaTrip_id = listaTrip_id.to_list()
print(listaTrip_id) """

In [30]:
""" resultados = []
dfs = []
for value in listaTrip_id:
    f = frequencies[(frequencies['trip_id'] == value)].reset_index()
    dfs.append(f)

# crie um ThreadPoolExecutor com 2 threads (pode ser ajustado conforme necessário)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    
    resultados_threads = [executor.submit(criacaoPartidas, df) for df in dfs]
               
    # colete os resultados das threads conforme eles estiverem prontos
    for resultado in concurrent.futures.as_completed(resultados_threads):
        resultados.append(resultado.result())

# concatene os resultados em um único dataframe
df_final = pd.concat(resultados) """

" resultados = []\ndfs = []\nfor value in listaTrip_id:\n    f = frequencies[(frequencies['trip_id'] == value)].reset_index()\n    dfs.append(f)\n\n# crie um ThreadPoolExecutor com 2 threads (pode ser ajustado conforme necessário)\nwith concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:\n    \n    resultados_threads = [executor.submit(criacaoPartidas, df) for df in dfs]\n               \n    # colete os resultados das threads conforme eles estiverem prontos\n    for resultado in concurrent.futures.as_completed(resultados_threads):\n        resultados.append(resultado.result())\n\n# concatene os resultados em um único dataframe\ndf_final = pd.concat(resultados) "

In [31]:
""" f = ['1017-10-0', '1012-10-0']
f1 = frequencies[(frequencies['trip_id'] == 1012-10-0)].reset_index() """
""" 
freq = []
f1 = frequencies[(frequencies['trip_id'] == '1017-10-0')].reset_index()
f2 = frequencies[(frequencies['trip_id'] == '1012-10-0')].reset_index()
freq.append(f1)
freq.append(f2)

resultados= []
threads = []
for f in freq:
    t = threading.Thread(target=criacaoPartidas, args=(f))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

for t in threads:
    resultados.append(t.result())
    print(resultados)
 """

"-------------------------------------------------"
""" 
threads = []
for idx,value in enumerate(listaTrip_id):
    f = frequencies[(frequencies['trip_id'] == value)].reset_index()
    t = threading.Thread(target=criacaoPartidas, args=(f))
    threads.append(t)
    t.start()

for t in threads:
    t.join() """



" \nthreads = []\nfor idx,value in enumerate(listaTrip_id):\n    f = frequencies[(frequencies['trip_id'] == value)].reset_index()\n    t = threading.Thread(target=criacaoPartidas, args=(f))\n    threads.append(t)\n    t.start()\n\nfor t in threads:\n    t.join() "

In [19]:
intervals = stop_times[(stop_times['trip_id']== '609F-21-0')]
print(intervals.count())

trip_id           49
arrival_time      49
departure_time    49
stop_id           49
stop_sequence     49
intervalos        49
dtype: int64


In [20]:
#intervals = stop_times[(stop_times['trip_id']== '609F-21-0')]
intervals = stop_times
intervals['intervalos'] = intervals['arrival_time'].diff().fillna('00:00:00')
intervals = intervals[['trip_id', 'stop_id', 'intervalos']]

In [35]:
new_stop_times = {'departure_id': [],
                               'trip_id': [],
                               'arrival_time':[],
                               'stop_id':[]}

for i in tqdm(range(len(partidas))):
    row = partidas.iloc[i]
    id = row['trip_id']
    rota = intervals.loc[(intervals['trip_id']==id)].reset_index(drop=True)
    rota_count = rota['trip_id'].count()
    tempo_partida = row['departure_time']
    tempo = row['departure_time']
    for i in range(rota_count):
        tempo += rota.loc[i, 'intervalos']
        
        new_stop_times['departure_id'].append(row['departure_id'])
        new_stop_times['trip_id'].append(row['trip_id'])
        new_stop_times['arrival_time'].append(tempo),
        new_stop_times['stop_id'].append(int(rota.loc[i,'stop_id']))

new_stop_times = pd.DataFrame(new_stop_times)

  2%|▊                                   | 4108/172844 [00:35<24:35, 114.38it/s]


KeyboardInterrupt: 

In [192]:
new_stop_times['hour'] = new_stop_times['arrival_time'].dt.hour
new = new_stop_times.loc[(new_stop_times['hour'] >= 7) & (new_stop_times['hour'] < 9)]
print(new)

       departure_id    trip_id        arrival_time   stop_id  hour
219  1012-10-0_6:24  1012-10-0 2023-03-01 07:02:00    301771     7
233  1012-10-0_6:36  1012-10-0 2023-03-01 07:00:42    301714     7
234  1012-10-0_6:36  1012-10-0 2023-03-01 07:02:36    301768     7
235  1012-10-0_6:36  1012-10-0 2023-03-01 07:04:30  30003015     7
236  1012-10-0_6:36  1012-10-0 2023-03-01 07:06:24  30003018     7
..              ...        ...                 ...       ...   ...
442  1012-10-0_8:48  1012-10-0 2023-03-01 08:51:48    301724     8
443  1012-10-0_8:48  1012-10-0 2023-03-01 08:53:42    301730     8
444  1012-10-0_8:48  1012-10-0 2023-03-01 08:55:36  30003042     8
445  1012-10-0_8:48  1012-10-0 2023-03-01 08:57:30  30003045     8
446  1012-10-0_8:48  1012-10-0 2023-03-01 08:59:24  30003043     8

[200 rows x 5 columns]
